In [7]:
import torch
from facenet_pytorch import InceptionResnetV1
from PIL import Image
from torchvision import transforms
import numpy as np
import os

# Загрузка модели FaceNet
model = InceptionResnetV1(pretrained='vggface2').eval()

# Функция для обработки изображений и извлечения эмбеддингов
def get_embedding(image_path):
    # Открытие изображения
    img = Image.open(image_path)
    # Преобразование изображения для модели
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img).unsqueeze(0)  # Добавляем batch dimension
    # Извлечение эмбеддинга
    with torch.no_grad():
        embedding = model(img_tensor)
    return embedding.squeeze().numpy()

# Функция для вычисления среднего эмбеддинга для набора изображений
def get_average_embedding(image_folder):
    embeddings = []
    for img_file in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_file)
        embedding = get_embedding(img_path)
        embeddings.append(embedding)
    return np.mean(embeddings, axis=0)

# Пути к папкам с изображениями
mother_folder = './Инна'  # Замените на путь к изображениям матери
father_folder = './Вадим'  # Замените на путь к изображениям отца
child_image = 'B14dos3.jpg'  # Замените на путь к изображению ребенка

# Извлечение средних эмбеддингов
mother_embedding = get_average_embedding(mother_folder)
father_embedding = get_average_embedding(father_folder)

# Эмбеддинг для ребенка
child_embedding = get_embedding(child_image)

# Функция для расчета косинусного расстояния
def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

# Расчет схожести
similarity_mother = cosine_similarity(child_embedding, mother_embedding)
similarity_father = cosine_similarity(child_embedding, father_embedding)

# Определение результата
if similarity_mother > similarity_father:
    print(f"Ребенок больше похож на мать (схожесть: {similarity_mother*100:.1f}%)")
else:
    print(f"Ребенок больше похож на отца (схожесть: {similarity_father*100:.1f}%)")


Ребенок больше похож на мать (схожесть: 46.2%)
